In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from likelihoodtools import LikelihoodGridScanner, AdaptiveLikelihoodScanner, GridFitter, FunctionFitter, GaussianLikelihoodModel
from likelihoodtools import make_1d_llh_plot, make_2d_llh_plot
from likelihoodtools import add_noise

#in this demo sin_model_data is the function f that generates the noise free data
#due to the parallelization it has to be defined outside the notebook. It only works if it is defined in the top level of a 
#module. For reasons. (That means lambdas or local functions in functions don't work)
from sinemodel import sin_model_data

In [ ]:
def make_experiment(theta, likelihood_model):
    
    data = likelihood_model.f(*theta)
    data = add_noise(data, likelihood_model)
    
    return data

Let's define some fitter. The fitter only needs to know the confidence levels. There is no fancy minimization done with these fitters. Their primary purpose is to find the confidence regions and not the maxima. GridFitter searches for the confidence levels as the first entry on a grid that is above the threshold. FunctionFitter searches for the confidence levels via root finding and only works if there is e.g. an interpolation function of the likelihood space not just a number of evaluations on a grid. I need to check if the FunctionFitter still works with latest changes and if it is actually a good choice.

In [ ]:
sigma_levels = [3, 2, 1]
fitter = GridFitter(sigma_levels=sigma_levels)
f_fitter = FunctionFitter(sigma_levels=sigma_levels)

Now let's define a model. We define a model with 4 parameters A, w, phase and DC. We assume gaussian noise with a standard deviation of 0.5 and use the imported function sin_model_data. If you want other noise you need a new child class of LikelihoodModel.

In [ ]:
ax_names = ['A','w','phase','DC']
noise_std = 0.5
model = GaussianLikelihoodModel(sin_model_data, noise_std, complex_data=False)

Now let's define a truth and let's look at a single experiment.

In [ ]:
truth = [2.0, 2., -1.5, 3.]

y  = make_experiment(truth, model)
plt.plot(y, label='single experiment')
plt.xlabel('x')
plt.ylabel('y')
plt.legend(loc='upper right')
plt.show()

Now we want to define a likelihood scanning grid. The list `delta` defines the scanning interval around the truth for each parameter. The nested list for the `DC` parameter defines an asymmetric scanning interval, i.e. the interval [truth-0.3, truth+0.4]. The list `n_eval` defines the number of grid points we want to have for the final scan along each axis. Since `n_eval` is zero for the `A` and the `DC` parameter this defines only a 2D scan in the `w-phase` subplane centered around the truth of the other dimensions. Both higher and lower dimensions possible but keep in mind the curse of dimensionality!

In [ ]:
delta = [0.6, 0.1, 0.5, [0.3,0.4]]
n_eval = [0, 201, 201, 0]

Now let's do a scan with the LikelihoodGridScanner, which does the scan using brute force. It does the evaluation in parallel by default and the result is interpolated by default if the dimensionality of the scan is 2 or smaller. The list `n` defines the number of grid points for evaluation of the expensive LLH function prior to interpolation. The final scan result is done by evaluating the interpolated result on the grid defined with `n_eval`. Calling the scanner for the data `y` returns a `LikelihoodScan` object that contains all the relevant information. The array containing the likelihood data is `llh_scan_2d.llh` and the interpolation function is `llh_scan_2d.llh_f`. The array can be reproduced with `llh_scan_2d.llh_f(llh_scan_2d.axes)`. Note that the performance here is not great for this very simple function because the parallelization overhead is too high.

In [ ]:
n = [0, 11, 11, 0]
llh_scanner_2d = LikelihoodGridScanner(truth, delta, n_eval, ax_names, model, n)
llh_scan_2d = llh_scanner_2d(y)

Now let's extract the errors. The `parameters_of_interest` defines that this is a 2D fit and no parameters are nuisance parameters. The fit result can be plotted with a function. Using the `name` parameter of the function saves the plot in a file with the provided name. Plotting functions are only available for 2D and 1D scans. The dimensionality is considered for the correct degrees of freedom of the chi2 that is used to convert a confidence level to a LLH threshold. However, I do realize now that I decided to use `sigma_levels` to define the confidence levels of the `fitter` object. For the 2D case 1 sigma of course is not 68% but 39%. Internally the `sigma_levels` are converted to a confidence level immediately, which is based on the 1D Gaussian distribution. So just do not expect sigma levels in the final result but the confidence level associated with the input sigma level in the 1D case. Treating the result as a confidence level the plot below shows the result correctly.

In [ ]:
fit_result_2d = fitter.get_best_fit_with_errors(llh_scan_2d,parameters_of_interest=2)
print(fit_result_2d.errors)
make_2d_llh_plot(fit_result_2d, name='fit_2d')

Let's look at the same LLH scan again, but this time we fit with the phase as nuisance parameter. By setting parameters of interest to 1 we assume only the first dimension is interesting, so we marginalize it. With `parameters_of_interest=n` it always marginalizes the first `n` dimensions. We plot the result in 1D.

In [ ]:
fit_result_1d = fitter.get_best_fit_with_errors(llh_scan_2d,parameters_of_interest=1)
print(fit_result_1d.errors)
make_1d_llh_plot(fit_result_1d, name='fit1d_phase_nuisance')

Alternatively, we can also get a 1D plot by doing a simple slice through the 2D scan using the `make_view` function of the `LikelihoodScan`. The argument is a tuple that is `None` for the axes that we want to keep and for the axes that we want to slice we provide the position by its value. By using `default` it defaults to using the truth value. Here we get one view along the `w` at the truth of the `phase` and one view along the `phase` axis at `w=2.05`. The `make_view` mechanism also works for higher dimensional scans.

In [ ]:
llh_scan_view = llh_scan_2d.make_view((None, 'default'))

fit_result_1d = fitter.get_best_fit_with_errors(llh_scan_view)
print(fit_result_1d.errors)
make_1d_llh_plot(fit_result_1d, name='fit1d_phase_view_w')

llh_scan_view = llh_scan_2d.make_view((2.05, None))

fit_result_1d = fitter.get_best_fit_with_errors(llh_scan_view)
print(fit_result_1d.errors)
make_1d_llh_plot(fit_result_1d, name='fit1d_phase_view_phase')

Finally if we want to save time we can also just do the scan in just 1 dimension in the first place. That means of course we assume the phase is known.

In [ ]:
#Again using fewer points pre interpolation. For these very parabolic maxima, 3 points are sufficient for a good interpolation.
n = [0, 3, 0, 0]
n_eval = [0, 201, 0, 0]

llh_scanner_1d = LikelihoodGridScanner(truth, delta, n_eval, ax_names, model, n)
llh_scan_1d = llh_scanner_1d(y)

fit_result_1d = fitter.get_best_fit_with_errors(llh_scan_1d,parameters_of_interest=1)
print(fit_result_1d.errors)
make_1d_llh_plot(fit_result_1d, name='fit1d_fixed_phase')

In Addition to the brute force `LikelihoodGridScanner` there is also the `AdaptiveLikelihoodScanner` that finds interesting regions of the likelihood space on its own. It is based on this package https://github.com/python-adaptive/adaptive . In theory the underlying package supports N dimensional functions as well, but I've only implemented it for 1D and 2D scans. The result is a `LikelihoodScan` again, so you can use it as in all the other examples before. For the example sine model in this demo it is pretty useless because the likelihood landscape can be understood very well with a uniform grid. The adaptive sampling algorithm adds some overhead so it is not efficient to evaluate very fast functions <50ms. Also in my observations in most 1D cases I got better results faster with a uniform grid, I think the likelihood needs very sharp features with unknown location for it to be more efficient in the 1D case. For the 2D case on the other hand it is absolutely worth it.

The parameter `debug` is optional and shows how many points were used to evaluate the likelihood and it makes a plot to show where it evaluated the likelihood. The parameter `loss_goal` determines the convergence criterion of the adaptive scan. I don't have a good guideline for how to set it. In this example the result is not quite smooth unless one uses a tiny loss goal which results in way more points than would be necessary on a uniform grid with interpolation. I guess this is a problem caused by the interpolation of irregular data points. This might be a concern but the example here makes it look worse than it actually is. The adaptive scanner shines for likelihood landscapes with narrow curved maxima in 2D. Even using interpolation these kinds of problems need extremely high resolutions on a regular grid.

In [ ]:
n_eval = [0, 201, 201, 0]

llh_scanner_adaptive = AdaptiveLikelihoodScanner(truth, delta, n_eval, ax_names, model, loss_goal=0.01, 
                                                    debug=True, cubic_spline=False)
llh_scan_ad = llh_scanner_adaptive(y)

In [ ]:
fit_result = fitter.get_best_fit_with_errors(llh_scan_ad, parameters_of_interest=2)
print(fit_result.errors)
make_2d_llh_plot(fit_result, name='fit_2d_adaptive')

fit_result = fitter.get_best_fit_with_errors(llh_scan_ad, parameters_of_interest=1)
print(fit_result.errors)
make_1d_llh_plot(fit_result, name='fit1d_phase_nuisance_adaptive')

llh_scan_view = llh_scan_ad.make_view((None, 'default'))
fit_result = fitter.get_best_fit_with_errors(llh_scan_view)
print(fit_result.errors)
make_1d_llh_plot(fit_result)

llh_scan_view = llh_scan_ad.make_view(('default', None))
fit_result = fitter.get_best_fit_with_errors(llh_scan_view)
print(fit_result.errors)
make_1d_llh_plot(fit_result)